### Specifying the function to be optimized

In [1]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1


### Getting Started

In [5]:
from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'x': (2,4), 'y': (-3,3)}
optimizer = BayesianOptimization(
  f=black_box_function,
  pbounds=pbounds,
  verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
  random_state=1
) 


In [6]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)


|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | -7.135    | 2.834     | 1.322     |
| 2         | -7.78     | 2.0       | -1.186    |
| 3         | -7.391    | 2.841     | 1.567     |
| 4         | -7.09     | 2.828     | 1.305     |
| 5         | -7.842    | 2.776     | -0.0644   |


In [7]:
print(optimizer.max)


{'target': -7.089552795751747, 'params': {'x': 2.8278543172354595, 'y': 1.3046190346063733}}


In [8]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))


Iteration 0: 
	{'target': -7.135455292718879, 'params': {'x': 2.8340440094051482, 'y': 1.3219469606529486}}
Iteration 1: 
	{'target': -7.779531005607566, 'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}}
Iteration 2: 
	{'target': -7.390502596642655, 'params': {'x': 2.8406441080806806, 'y': 1.5667835988004315}}
Iteration 3: 
	{'target': -7.089552795751747, 'params': {'x': 2.8278543172354595, 'y': 1.3046190346063733}}
Iteration 4: 
	{'target': -7.841680406240954, 'params': {'x': 2.7764615902126666, 'y': -0.06439722111376445}}


### Changing bounds

In [9]:
optimizer.set_bounds(new_bounds={"x": (-2,3)})
optimizer.maximize(
  init_points=0,
  n_iter=5,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 6         | -6.503    | 2.738     | 1.087     |
| 7         | -4.752    | 2.398     | 0.9646    |
| 8         | -3.036    | 2.009     | 0.9578    |
| 9         | -0.4794   | 1.196     | 1.222     |
| 10        | 0.7132    | -0.2876   | 1.452     |


### Guiding the optimization

It is often the case that we have an idea of regions of the parameter space where the maximum of our function might lie. For these situations the BayesianOptimization object allows the user to specify specific points to be probed. By default these will be explored lazily (lazy=True), meaning these points will be evaluated only the next time you call maximize. This probing process happens before the gaussian process takes over.

Parameters can be passed as dictionaries such as below:

In [10]:
optimizer.probe(
  params = {"x": 0.5, "y": 0.7},
  lazy=True,
)
print(optimizer.space.keys)
optimizer.probe(
  params =[-0.3, 0.1],
  lazy=True,
)
optimizer.maximize(init_points=0, n_iter=0)

['x', 'y']
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 11        | 0.66      | 0.5       | 0.7       |
| 12        | 0.1       | -0.3      | 0.1       |


### Saving, loading and restarting

By default you can follow the progress of your optimization by setting verbose>0 when instantiating the BayesianOptimization object. If you need more control over logging/alerting you will need to use an observer. For more information about observers checkout the advanced tour notebook. Here we will only see how to use the native JSONLogger object to save to and load progress from files.

### Saving progress

In [11]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events


In [12]:
logger = JSONLogger(path="./logs.log")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
optimizer.maximize(
    init_points=2,
    n_iter=3,
)


|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 13        | -12.48    | -1.266    | -2.446    |
| 14        | -3.854    | -1.069    | -0.9266   |
| 15        | -7.0      | -2.0      | 3.0       |
| 16        | -3.05     | -1.965    | 0.5651    |
| 17        | -3.211    | 0.4597    | 3.0       |


### Loading progress

Naturally, if you stored progress you will be able to load that onto a new instance of BayesianOptimization. The easiest way to do it is by invoking the load_logs function, from the util submodule.

In [13]:
from bayes_opt.util import load_logs
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))
load_logs(new_optimizer, logs=["./logs.log"]);
print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))
new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)


0
New optimizer is now aware of 5 points.
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 0.0603    | 0.7596    | 0.3977    |
| 2         | -1.578    | 1.585     | 1.253     |
| 3         | -4.79     | 1.888     | -0.4925   |


/Users/anjalisuman/anaconda3/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:225: UserWarning: 
Data point [-1.26622055 -2.44596843] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)
/Users/anjalisuman/anaconda3/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:225: UserWarning: 
Data point [-2.  3.] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)
/Users/anjalisuman/anaconda3/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:225: UserWarning: 
Data point [0.45968581 3.        ] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)


| 4         | 0.9607    | 0.08382   | 1.18      |
| 5         | 0.3744    | -0.4092   | 0.3231    |
| 6         | 0.4804    | -0.6529   | 1.305     |
| 7         | 0.9304    | 0.0189    | 0.7368    |
| 8         | 0.6895    | 0.5539    | 1.061     |
| 9         | 0.866     | -0.3598   | 0.9328    |
| 10        | 0.7173    | -0.1128   | 1.52      |
